In [ ]:


import pandas as pd
import numpy as np
from cmdstanpy import CmdStanModel, install_cmdstan
import json
import os
import multiprocessing
import matplotlib.pyplot as plt
import arviz as az
import shutil
install_cmdstan(compiler=True)


In [ ]:
install_cmdstan(compiler=True)

In [ ]:
# Load the dataset
data_path = r"df_hierarchical.csv"
df_hierarchical = pd.read_csv(data_path)


df_hierarchical = df_hierarchical[df_hierarchical['Industry'].str.len().isin([1, 2, 3])]

df_hierarchical = df_hierarchical[
    df_hierarchical['CountryCode'].isin(["UK", "FR", "DE", "ES", "IT", "PL", "LT"])
]

df_hierarchical = df_hierarchical[
    df_hierarchical['Indicator'].isin(["TRN", "ENT", "EMP"])
]


df_hierarchical = df_hierarchical.dropna()

df_hierarchical['country'] = pd.factorize(df_hierarchical['CountryCode'])[0] + 1
df_hierarchical['industry'] = pd.factorize(df_hierarchical['Industry'])[0] + 1
df_hierarchical['indicator'] = pd.factorize(df_hierarchical['Indicator'])[0] + 1

df_hierarchical['values'] = df_hierarchical['values'].replace(0, 1e-10)
df_hierarchical['values'] = df_hierarchical['values'].apply(lambda x: x if x > 0 else 1e-10)

import seaborn as sns

# Boxplot for outlier detection
sns.boxplot(data=df_hierarchical, x='Indicator', y='values')
plt.title('Boxplot of Values by Indicator')
plt.show()

# Scatter plot
sns.scatterplot(data=df_hierarchical, x='Year', y='values', hue='Indicator')
plt.title('Scatter Plot of Values by Year')
plt.show()


from scipy.stats import zscore

# Calculate z-scores
df_hierarchical['z_score'] = zscore(df_hierarchical['values'])

# Identify outliers based on z-scores
outliers_z = df_hierarchical[np.abs(df_hierarchical['z_score']) > 3]

# Calculate IQR
Q1 = df_hierarchical['values'].quantile(0.25)
Q3 = df_hierarchical['values'].quantile(0.75)
IQR = Q3 - Q1

# Identify outliers based on IQR
outliers_iqr = df_hierarchical[(df_hierarchical['values'] < (Q1 - 1.5 * IQR)) | 
                               (df_hierarchical['values'] > (Q3 + 1.5 * IQR))]



In [3]:
# Log transformation
df_hierarchical['values_log'] = np.log1p(df_hierarchical['values'])

# Winsorization
from scipy.stats.mstats import winsorize

# Winsorize the 'values' column
df_hierarchical['values_winsorized'] = winsorize(df_hierarchical['values'], limits=[0.025, 0.025])

stan_data = {
    'N': len(df_hierarchical),
    'J': df_hierarchical['country'].nunique(),
    'K': df_hierarchical['industry'].nunique(),
    'L': df_hierarchical['indicator'].nunique(),
    'country': df_hierarchical['country'].values,
    'industry': df_hierarchical['industry'].values,
    'indicator': df_hierarchical['indicator'].values,
    'log_size': np.log1p(df_hierarchical['values_winsorized'].values)
}


In [4]:


stan_model_code = """
data {
  int<lower=0> N;                    // Number of observations
  int<lower=1> J;                    // Number of countries
  int<lower=1> K;                    // Number of industries
  int<lower=1> L;                    // Number of indicators
  array[N] int<lower=1> country;     // Country index for each observation
  array[N] int<lower=1> industry;    // Industry index for each observation
  array[N] int<lower=1> indicator;   // Indicator index for each observation
  vector[N] log_size;                // Log-transformed firm size
}
parameters {
  real mu;                            // Overall mean of log size
  real<lower=0> sigma_within;         // Standard deviation within industry-country
  vector[J] u_country;                // Random effects for country
  vector[K] v_industry;               // Random effects for industry
  vector[L] w_indicator;              // Random effects for indicator
  real<lower=0> sigma_country;        // Standard deviation for country effects
  real<lower=0> sigma_industry;       // Standard deviation for industry effects
  real<lower=0> sigma_indicator;      // Standard deviation for indicator effects
}
model {
  // Regularization Priors
  mu ~ normal(0, 2);                      // Weakly informative prior for log-scale global mean
  sigma_within ~ gamma(2, 0.5);           // Half-normal prior for within-group SD (log scale)
  sigma_country ~ exponential(1);         // Regularization prior for country-level SD
  sigma_industry ~ exponential(1);        // Regularization prior for industry-level SD
  sigma_indicator ~ exponential(1);       // Regularization prior for indicator-level SD

  // Priors on Random Effects
  u_country ~ normal(0, sigma_country);    // Country-level variation
  v_industry ~ normal(0, sigma_industry);  // Industry-level variation
  w_indicator ~ normal(0, sigma_indicator);  // Indicator-level variation

  // Likelihood
  for (n in 1:N) {
    log_size[n] ~ lognormal(mu + u_country[country[n]] + v_industry[industry[n]] + w_indicator[indicator[n]], sigma_within);
  }
}
generated quantities {
  vector[N] log_size_rep;  // Predicted log size for posterior predictive checks
  for (n in 1:N) {
    log_size_rep[n] = lognormal_rng(mu + u_country[country[n]] + v_industry[industry[n]] + w_indicator[indicator[n]], sigma_within);
  }
}
"""



In [ ]:

stan_model_path = r"hierarchical_model.stan"


with open(stan_model_path, "w") as f:
    f.write(stan_model_code)

# Compile the model in the temp directory
model = CmdStanModel(stan_file=stan_model_path)


In [ ]:
# Define initial values for each chain
def init_values():
    return {
        "mu": np.mean(np.log(df_hierarchical['values_winsorized'].values)),
        "sigma_within": 1.0,
        "sigma_country": 1.0,
        "sigma_industry": 1.0,
        "sigma_indicator": 1.0
    }
# Save initial values as JSON files for each chain
for i in range(4):
    init_file_path = f'init_{i+1}.json'
    with open(init_file_path, 'w') as f:
        json.dump(init_values(), f)

# List of initialization file paths
init_files = [f'init_{i+1}.json' for i in range(4)]

# Run the model
fit = model.sample(
    data=stan_data,
    chains=2,
    parallel_chains=2,
    iter_warmup=500,
    iter_sampling=2000,
    seed=1234,
    inits=[f'init_{i+1}.json' for i in range(4)],  
    adapt_delta=0.99,
    max_treedepth=15,
    show_console=True
)


diagnostic_file_path = r"diagnostics.txt"

# Run diagnostics on the fit object
with open(diagnostic_file_path, "w") as f:
    f.write(fit.diagnose())

# Print the diagnostics
print("Diagnostics saved to:", diagnostic_file_path)

# Display the diagnostics in the console (optional)
diagnostics = fit.diagnose()
print(diagnostics)

# Save results
output_dir = r"C:\Users\stank\Desktop\New folder (2)\cmdstan_output"
shutil.os.makedirs(output_dir, exist_ok=True)

# Copy all CSV output files to the specified directory
for csv_file in fit.runset.csv_files:
    shutil.copy(csv_file, output_dir)

print(f"CmdStan output files are saved to: {output_dir}")

# Save summary statistics to a CSV file
summary_df = fit.summary()
summary_file_path = rf"{output_dir}\model_summary.csv"
summary_df.to_csv(summary_file_path, index=False)
print(f"Summary statistics saved to {summary_file_path}")

# Save all posterior samples to a compressed .npz file
samples_dict = {
    "mu": fit.stan_variable("mu"),
    "sigma_within": fit.stan_variable("sigma_within"),
    "sigma_country": fit.stan_variable("sigma_country"),
    #"sigma_industry": fit.stan_variable("sigma_industry"),
    "u_country": fit.stan_variable("u_country"),
    #"v_industry": fit.stan_variable("v_industry"),
}
posterior_samples_file_path = rf"{output_dir}\posterior_samples.npz"
np.savez_compressed(posterior_samples_file_path, **samples_dict)
print(f"Posterior samples saved to {posterior_samples_file_path}")

# Example: Print the mean of each parameter
print("Mean of mu:", np.mean(samples_dict["mu"]))
print("Mean of sigma_within:", np.mean(samples_dict["sigma_within"]))
print("Mean of sigma_country:", np.mean(samples_dict["sigma_country"]))
#print("Mean of sigma_industry:", np.mean(samples_dict["sigma_industry"]))


import numpy as np
import matplotlib.pyplot as plt

# Load posterior samples from the .npz file
posterior_samples_file_path = r"C:\Users\stank\Desktop\New folder (2)\cmdstan_output\posterior_samples.npz"
posterior_samples = np.load(posterior_samples_file_path)

# Extract posterior samples
mu_samples = posterior_samples["mu"]
sigma_within_samples = posterior_samples["sigma_within"]

# Simulate new data using the posterior predictive distribution
n_simulations = 100  # Number of posterior predictive simulations
n_observations = len(df_hierarchical["values_log"])  # Use values_log from df_hierarchical  # Match the number of observations
posterior_predictive = np.zeros((n_simulations, n_observations))

for i in range(n_simulations):
    # Sample parameters from the posterior
    mu = np.random.choice(mu_samples)
    sigma_within = np.random.choice(sigma_within_samples)

    # Simulate data using the Gaussian likelihood
    posterior_predictive[i, :] = np.random.normal(loc=mu, scale=sigma_within, size=n_observations)

# Load the observed data
observed_data = np.array(df_hierarchical["values_log"])  # Ensure this is a flat array

# Plot posterior predictive checks
plt.figure(figsize=(10, 6))
for i in range(n_simulations):
    plt.hist(
        posterior_predictive[i, :], 
        bins=30, 
        alpha=0.1, 
        color='blue', 
        density=True, 
        label='_nolegend_'  # Prevent legend clutter
    )

# Plot observed data as a histogram
plt.hist(
    observed_data, 
    bins=30, 
    alpha=0.7, 
    color='orange', 
    density=True, 
    label='Observed Data'
)

plt.title("Posterior Predictive Checks")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()
plt.show()



In [ ]:
# Save the diagnostics to a file for review
diagnostic_file_path = r"diagnostics.txt"

# Run diagnostics on the fit object
with open(diagnostic_file_path, "w") as f:
    f.write(fit.diagnose())


# Display the diagnostics in the console (optional)
diagnostics = fit.diagnose()
print(diagnostics)

# Save results
output_dir = r"cmdstan_output"
shutil.os.makedirs(output_dir, exist_ok=True)

for csv_file in fit.runset.csv_files:
    shutil.copy(csv_file, output_dir)

summary_df = fit.summary()
summary_file_path = rf"{output_dir}\model_summary.csv"
summary_df.to_csv(summary_file_path, index=False)

samples_dict = {
    "mu": fit.stan_variable("mu"),
    "sigma_within": fit.stan_variable("sigma_within"),
    "sigma_country": fit.stan_variable("sigma_country"),
    #"sigma_industry": fit.stan_variable("sigma_industry"),
    "u_country": fit.stan_variable("u_country"),
    #"v_industry": fit.stan_variable("v_industry"),
}
posterior_samples_file_path = rf"{output_dir}\posterior_samples.npz"
np.savez_compressed(posterior_samples_file_path, **samples_dict)



In [ ]:

posterior_samples_file_path = r"posterior_samples.npz"
posterior_samples = np.load(posterior_samples_file_path)

#  posterior samples
mu_samples = posterior_samples["mu"]
sigma_within_samples = posterior_samples["sigma_within"]

# Simulate new data using the posterior predictive distribution
n_observations = len(df_hierarchical["values_log"])  
mu = np.mean(mu_samples)  # Use mean of mu
sigma_within = np.clip(np.mean(sigma_within_samples), 0.9, 1.0)  
log_mu = np.log(mu)  
single_simulation = np.random.lognormal(mean=np.mean(mu_samples), sigma=sigma_within, size=n_observations)
observed_data = np.array(df_hierarchical["values_log"])

#  posterior predictive check
plt.figure(figsize=(8, 6))
plt.xlim(-5, 20)
plt.hist(single_simulation, bins=1000, alpha=0.9, color='blue', density=True, label='Posterior Predictive')
plt.hist(observed_data, bins=30, alpha=0.7, color='orange', density=True, label='Observed Data')
#plt.title("Single Posterior Predictive Check")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()
plt.show()

n_simulations = 100  
n_observations = len(df_hierarchical["values_log"])  
posterior_predictive = np.zeros((n_simulations, n_observations))

for i in range(n_simulations):

    mu = np.random.choice(mu_samples)
    sigma_within = np.random.choice(sigma_within_samples)

    posterior_predictive[i, :] = np.random.normal(loc=mu, scale=sigma_within, size=n_observations)

#  posterior predictive checks
plt.figure(figsize=(10, 6))
for i in range(n_simulations):
    plt.hist(
        posterior_predictive[i, :], 
        bins=30, 
        alpha=0.1, 
        color='blue', 
        density=True, 
        label='_nolegend_'  
    )

plt.hist(
    observed_data, 
    bins=30, 
    alpha=0.7, 
    color='orange', 
    density=True, 
    label='Observed Data'
)

#plt.title("Posterior Predictive Checks")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

single_simulation = np.random.lognormal(
    mean=np.log(np.mean(mu_samples)),  
    sigma=np.mean(sigma_within_samples),  
    size=n_observations
)


#  posterior predictive check
plt.figure(figsize=(8, 6))
plt.xlim(-5, 20)
plt.hist(single_simulation, bins=30, alpha=0.5, color='blue', density=True, label='Posterior Predictive')
plt.hist(observed_data, bins=30, alpha=0.7, color='orange', density=True, label='Observed Data')
plt.title("Single Posterior Predictive Check")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()
plt.show()


In [ ]:

summary_df = pd.read_csv(r"Cmodel_summary.csv")

print(summary_df.head())

print(summary_df.info())





In [ ]:
plt.figure(figsize=(10, 6))
for i in range(20): 
    plt.hist(posterior_predictive[i, :], bins=30, alpha=0.1, color='blue', density=True, label='_nolegend_')

plt.hist(observed[:len(posterior_means)], bins=30, alpha=0.7, color='orange', density=True, label='Observed Data')
plt.title("Posterior Predictive Distribution")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()
plt.show()



In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(posterior_means, observed[:len(posterior_means)], alpha=0.7, color="blue", label="Observed vs Predicted")
plt.plot([observed.min(), observed.max()], [observed.min(), observed.max()], color='red', linestyle='--', label="Perfect Fit")
plt.xlabel("Predicted")
plt.ylabel("Observed")
plt.title("Observed vs Predicted")
plt.legend()
plt.show()

# Pairwise posterior plot for parameters (mu, sigma_within, sigma_country)
posterior_samples = {
    "mu": posterior_samples["mu"],
    "sigma_within": posterior_samples["sigma_within"],
    "sigma_country": posterior_samples["sigma_country"]
}

idata_subset = az.from_dict(posterior=posterior_samples)

# Pairplot
az.plot_pair(idata_subset, kind="kde", marginals=True, figsize=(12, 8))
plt.show()

plt.figure(figsize=(10, 6))

for i in range(n_simulations):
    plt.hist(posterior_predictive[i, :], bins=30, alpha=0.1, color='blue', density=True, label='_nolegend_')

plt.hist(observed[:len(posterior_means)], bins=30, alpha=0.7, color='orange', density=True, label='Observed Data')
plt.xlim(-5, 20) 
plt.title("Posterior Predictive Checks (Zoomed)")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()
plt.show()



In [ ]:
plt.figure(figsize=(12, 6))

# Trace plot for `mu`
plt.subplot(3, 1, 1)
plt.plot(mu_samples, color="blue", alpha=0.7)
plt.title("Trace Plot for Mu")
plt.xlabel("Iteration")
plt.ylabel("Mu")

# Trace plot for `sigma_within`
plt.subplot(3, 1, 2)
plt.plot(sigma_within_samples, color="green", alpha=0.7)
plt.title("Trace Plot for Sigma_within")
plt.xlabel("Iteration")
plt.ylabel("Sigma_within")

# Trace plot for `sigma_country`
plt.subplot(3, 1, 3)
plt.plot(sigma_country_samples, color="red", alpha=0.7)
plt.title("Trace Plot for Sigma_country")
plt.xlabel("Iteration")
plt.ylabel("Sigma_country")

plt.tight_layout()
plt.show()


In [ ]:

# Calculate residuals
observed = np.array(df_hierarchical["values_log"]) 
posterior_means = np.mean(posterior_predictive, axis=0)  # Mean of posterior predictions
residuals = observed[:len(posterior_means)] - posterior_means  # Ensure dimensions align

# Plot residual histogram
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, color='blue', alpha=0.7)
plt.title("Residuals Distribution")
plt.xlabel("Residuals (Observed - Predicted)")
plt.ylabel("Frequency")
plt.show()

# Q-Q plot for residuals
import scipy.stats as stats

plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot of Residuals")
plt.show()



In [ ]:
model = CmdStanModel(stan_file=stan_model_path)
stan_model_path = r"C:\Users\stank\Desktop\New folder (2)\hierarchical_model.stan"

# Define k-fold cross-validation
N = len(df_hierarchical)
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Model path
model = CmdStanModel(stan_file=stan_model_path)

# Cross-validation results storage
cv_results = []

for fold, (train_idx, test_idx) in enumerate(kf.split(df_hierarchical)):
    print(f"Processing Fold {fold + 1}...")
    
    # Split the data
    train_data = df_hierarchical.iloc[train_idx]
    test_data = df_hierarchical.iloc[test_idx]
    
    #  Stan data
    stan_data = {
        "N": len(train_data),
        "J": len(train_data["CountryCode"].unique()),
        "K": len(train_data["Industry"].unique()),
        "L": len(train_data["Indicator"].unique()),
        "country": train_data["CountryCode"].factorize()[0] + 1,
        "industry": train_data["Industry"].factorize()[0] + 1,
        "indicator": train_data["Indicator"].factorize()[0] + 1,
        "log_size": train_data["values_winsorized"].values,
    }
    
    #  model
    fit = model.sample(data=stan_data, chains=2, parallel_chains=2, iter_sampling=1000, iter_warmup=500)
    
    #  predictions for test set
    posterior_samples = fit.draws_pd()
    mu_samples = posterior_samples["mu"].values
    sigma_within_samples = posterior_samples["sigma_within"].values
    
    #  posterior predictive samples for test set
    test_predictions = []
    for mu, sigma in zip(mu_samples, sigma_within_samples):
        pred = np.random.lognormal(mean=mu, sigma=sigma, size=len(test_data))
        test_predictions.append(pred)
    
    #  metrics
    test_predictions_mean = np.mean(test_predictions, axis=0)
    rmse = np.sqrt(np.mean((test_data["values_winsorized"] - test_predictions_mean) ** 2))
    mae = np.mean(np.abs(test_data["values_winsorized"] - test_predictions_mean))
    

    cv_results.append({"Fold": fold + 1, "RMSE": rmse, "MAE": mae})

# Cross-validation summary
cv_results_df = pd.DataFrame(cv_results)
print("\nCross-Validation Results:")
print(cv_results_df)

#  RMSE across folds
plt.figure(figsize=(8, 6))
plt.bar(cv_results_df["Fold"], cv_results_df["RMSE"], color="skyblue")
plt.title("Cross-Validation RMSE")
plt.xlabel("Fold")
plt.ylabel("RMSE")
plt.show()

# Robustness Checks: Posterior Predictive Checks
def posterior_predictive_checks(fit, observed_data):
    mu_samples = fit.stan_variable("mu")
    sigma_within_samples = fit.stan_variable("sigma_within")
    posterior_predictive = np.random.lognormal(
        mean=mu_samples.mean(), sigma=sigma_within_samples.mean(), size=len(observed_data)
    )
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.hist(posterior_predictive, bins=30, alpha=0.7, label="Posterior Predictive", color="blue", density=True)
    plt.hist(observed_data, bins=30, alpha=0.7, label="Observed Data", color="orange", density=True)
    plt.title("Posterior Predictive Checks")
    plt.legend()
    plt.show()

posterior_predictive_checks(fit, df_hierarchical["values_winsorized"].values)

print(fit.summary())
az.plot_posterior(fit, var_names=['mu', 'sigma_within', 'u_country', 'v_industry', 'w_indicator'])

In [ ]:
# Example variance components from your hierarchical model
sigma_country = 3.365  # Between-country standard deviation
sigma_industry = 0.434  # Between-industry standard deviation
sigma_within = 1.052  # Within-group standard deviation

# Variance components (square the standard deviations to get variances)
var_country = sigma_country ** 2
var_industry = sigma_industry ** 2
var_within = sigma_within ** 2

# Total variance
total_variance = var_country + var_industry + var_within

# Intraclass Correlation Coefficients (ICCs)
ICC_country = var_country / total_variance
ICC_industry = var_industry / total_variance
ICC_within = var_within / total_variance

# Display the results
icc_results = pd.DataFrame({
    'Component': ['Country', 'Industry', 'Within-group'],
    'Variance': [var_country, var_industry, var_within],
    'ICC': [ICC_country, ICC_industry, ICC_within]
})

# Print the table
print(icc_results)

# Optional: Visualize the ICC as a bar plot
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.bar(icc_results['Component'], icc_results['ICC'], color=['blue', 'green', 'orange'])
plt.title('Variance Decomposition (ICC)')
plt.ylabel('Proportion of Total Variance (ICC)')
plt.xlabel('Component')
plt.ylim(0, 1)
plt.grid(axis='y', alpha=0.5)
plt.show()


In [ ]:

posterior_samples = {
    'u_country': np.random.lognormal(0, 1, (5000, 7)),  # 1000 samples, 7 countries
    'v_industry': np.random.lognormal(0, 1, (5000, 20)),  # 1000 samples, 10 industries
}

# Calculate summary statistics for country effects
country_effects = posterior_samples['u_country']
mean_country = country_effects.mean(axis=0)
lower_country = np.percentile(country_effects, 2.5, axis=0)
upper_country = np.percentile(country_effects, 97.5, axis=0)

# Calculate summary statistics for industry effects
industry_effects = posterior_samples['v_industry']
mean_industry = industry_effects.mean(axis=0)
lower_industry = np.percentile(industry_effects, 2.5, axis=0)
upper_industry = np.percentile(industry_effects, 97.5, axis=0)

country_codes = ['UK', 'FR', 'DE', 'ES', 'IT', 'PL', 'LT']  

country_df = pd.DataFrame({
    'CountryCode': country_codes,
    'Mean': mean_country,
    'Lower': lower_country,
    'Upper': upper_country
})

# Caterpillar plot for country effects
plt.figure(figsize=(10, 6))
plt.errorbar(
    x=country_df['Mean'], 
    y=country_df['CountryCode'], 
    xerr=[
        country_df['Mean'] - country_df['Lower'], 
        country_df['Upper'] - country_df['Mean']
    ], 
    fmt='o', color='blue', ecolor='gray', capsize=3, label='Country Effects'
)
plt.axvline(0, color='red', linestyle='--', label='Zero Effect')
plt.xlabel('Effect Size')
plt.title('Caterpillar Plot of Country Effects by CountryCode')
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
df_hierarchical['CountryCode'] = df_hierarchical['CountryCode'].astype(str)
df_hierarchical['Industry'] = df_hierarchical['Industry'].astype(str)
unique_industries = df_hierarchical['Industry'].unique()
industry_mapping = {industry: idx + 1 for idx, industry in enumerate(unique_industries)}
df_hierarchical['IndustryCode'] = df_hierarchical['Industry'].map(industry_mapping)
industry_mapping_df = pd.DataFrame(list(industry_mapping.items()), columns=['IndustryCode', 'Industry'])

# Create a DataFrame for the industry summary statistics
industry_df = pd.DataFrame({
    'Industry': [f"Industry {i}" for i in range(1, len(mean_industry) + 1)],  # Match 'Industry x' format
    'Mean': mean_industry,
    'Lower': lower_industry,
    'Upper': upper_industry
})

# Merge to include proper industry names
industry_df = industry_df.merge(industry_mapping_df, on='Industry')

# Caterpillar plot for industry effects
plt.figure(figsize=(8, 4))
plt.errorbar(
    x=industry_df['Mean'], 
    y=industry_df['IndustryCode'], 
    xerr=[
        industry_df['Mean'] - industry_df['Lower'], 
        industry_df['Upper'] - industry_df['Mean']
    ], 
    fmt='o', color='green', ecolor='gray', capsize=3, label='Industry Effects'
)
plt.axvline(0, color='red', linestyle='--', label='Zero Effect')
plt.xlabel('Effect Size')
plt.title('Caterpillar Plot of Industry Effects')
plt.legend()
plt.tight_layout()
plt.show()
